In [ ]:
import pandas as pd
import os
import dbconnect # 제공해주신 DB 연결 모듈 사용
import sys


# 파일 경로 설정 (사용자 환경)

BASE_DIR = r"C:\Users\Admin\Desktop\hdc2\00. 프로젝트\02. 프로젝트 구축"
CSV_DIR = os.path.join(BASE_DIR, "csv")

# 매핑 파일 경로
MAPPING_FILE = os.path.join(BASE_DIR, "station_master_mapping.csv")

# 처리할 파일 목록 (순서대로 22 -> 23 -> 24)
TARGET_FILES = [
    os.path.join(CSV_DIR, "서울교통공사_역별 일별 시간대별 노인 승하차인원_2022.csv"),
    os.path.join(CSV_DIR, "서울교통공사_일별 역별 시간대별 노인 승하차인원_2023.csv"),
    os.path.join(CSV_DIR, "서울교통공사_역별 일별 시간대별 노인 승하차인원_2024.csv")
]

In [ ]:
# 호선 매핑 데이터 로드 함수

def load_mapping_table():
    
    df_map = pd.read_csv(MAPPING_FILE, encoding='utf-8')
    
    # Merge를 위해 stnCd를 문자열로 통일
    df_map['stnCd'] = df_map['stnCd'].astype(str).str.strip() # str.strip()는 공백(스페이스나 탭)을 제거
    
    # 필요한 컬럼만 리턴 (역코드, 호선명)
    return df_map[['stnCd', 'lineNm']]

In [ ]:
# 데이터 변환용 함수 정의


# 메인 함수 정의 

def process_file(file_path, df_mapping):
    
    # (1) CSV 로드
    try:
        df_raw = pd.read_csv(file_path, encoding='utf-8')
    except:
        df_raw = pd.read_csv(file_path, encoding='cp949')
        
    # (2) 컬럼명 매핑 (API에서 가져온 데이터 컬럼 이름으로 변경)
    rename_map = {
        '수송일자': 'pasngDe',
        '역번호': 'stnCd',
        '역명': 'stnNm',
        '승하차구분': 'type'
    }
    df_raw.rename(columns=rename_map, inplace=True)
    
    # (3) [Mapping] 호선명 채우기
    # 원본 CSV의 stnCd도 문자열로 변환하여 매칭 확률 높임
    df_raw['stnCd'] = df_raw['stnCd'].astype(str).str.strip()

    # Left Join 수행
    df_merged = pd.merge(df_raw, df_mapping, on='stnCd', how='left')
    
    # 매핑 안 된 역은 '정보없음' 처리
    df_merged['lineNm'] = df_merged['lineNm'].fillna('정보없음')
    
    # (4) Melt
    # id_vars에 lineNm 포함
    id_vars = ['pasngDe', 'stnCd', 'stnNm', 'type', 'lineNm']
        
    # 시간대 컬럼 자동 탐지
    value_vars = []  # 1. 빈 리스트를 하나 만듭니다.

    for col in df_merged.columns:      # 2. 전체 컬럼 이름을 하나씩 꺼내서 확인합니다.
        if '시간대' in col:            # 3. 만약 이름 안에 '시간대'라는 글자가 있다면?
            value_vars.append(col)     # 4. 리스트에 추가합니다.
            
    
    df_melted = df_merged.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='temp_hr',
        value_name='count'
    )
    
    # (5) 시간대 변환
    def convert_hour(hr_str):
        if '06시간대이전' in hr_str: return 5
        elif '24시간대이후' in hr_str: return 0
        else:
            try:
                return int(hr_str.split('-')[0])
            except:
                return -1

    df_melted['pasngHr'] = df_melted['temp_hr'].apply(convert_hour)
    
    # (6) Pivot (승/하차 분리)
    df_pivot = df_melted.pivot_table(
        index=['pasngDe', 'pasngHr', 'lineNm', 'stnCd', 'stnNm'],
        columns='type',
        values='count',
        fill_value=0
    ).reset_index()
    
    df_pivot.rename(columns={'승차': 'rideNope', '하차': 'gffNope'}, inplace=True)
    df_pivot.columns.name = None
    
    # (7) 메타데이터 추가
    df_pivot['UserGroup'] = '노인/약자'
    df_pivot['trnscdUserSeCd'] = '06'
    # 날짜 포맷 정리 (2022-01-01 -> 20220101)
    df_pivot['pasngDe'] = df_pivot['pasngDe'].astype(str).str.replace('-', '')
    
    # 최종 컬럼 순서
    target_cols = [
        'UserGroup', 'pasngDe', 'pasngHr', 'lineNm', 
        'stnCd', 'stnNm', 'trnscdUserSeCd', 
        'rideNope', 'gffNope'
    ]
    
    return df_pivot[target_cols]


# 적재 함수 정의

def insert_to_db(conn, df):
    cursor = conn.cursor()
    
    # 1. 사용할 컬럼 목록을 리스트로 명확히 정의합니다.
    # (DataFrame에서 이 순서대로 데이터를 뽑아냅니다)
    columns = [
        'UserGroup', 'pasngDe', 'pasngHr', 'lineNm', 
        'stnCd', 'stnNm', 'trnscdUserSeCd', 'rideNope', 'gffNope'
    ]
    
    # 2. SQL 쿼리 작성
    # 테이블명에 하이픈(-)이 있으므로 반드시 백틱(`)으로 감싸줍니다.
    sql = """
    INSERT INTO `subway_traffic_log_senior_22-24`
    (UserGroup, pasngDe, pasngHr, lineNm, stnCd, stnNm, trnscdUserSeCd, rideNope, gffNope)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        rideNope = VALUES(rideNope),
        gffNope = VALUES(gffNope)
    """
    
    # 3. 데이터프레임에서 위에서 정한 컬럼들만 뽑아 값을 가져옵니다.
    # raw_data는 리스트의 리스트 형태가 됩니다.
    raw_data = df[columns].values
    
    count = 0
    
    # 4. 한 줄씩 꺼내서 반복문(Loop) 실행
    for row in raw_data:
        try:
            # row는 하나의 행 데이터입니다. (인덱스 0부터 8까지 존재)
            # 안전하게 넣기 위해 숫자는 int()로 명확히 변환해줍니다.
            data = (
                row[0],          # UserGroup
                row[1],          # pasngDe
                int(row[2]),     # pasngHr (시간은 숫자)
                row[3],          # lineNm
                row[4],          # stnCd
                row[5],          # stnNm
                row[6],          # trnscdUserSeCd
                int(row[7]),     # rideNope (인원수는 숫자)
                int(row[8])      # gffNope (인원수는 숫자)
            )
            
            # 한 줄 실행 (여기서 에러가 나면 해당 줄만 건너뜀)
            cursor.execute(sql, data)
            count += 1
            
        except Exception as e:
            # 어떤 데이터에서 에러가 났는지 출력해서 확인 가능
            print(f"에러: {e}")
            print(f"문제 데이터: {row}")
            
    # 5. 모든 반복이 끝나면 최종 저장(Commit)
    try:
        conn.commit()
        print(f"총 {count}건 저장 완료")
        return count
    except Exception as e:
        print(f"에러: {e}")
        conn.rollback()
        return 0

In [ ]:
# 메인 실행 블록


# 1. 매핑 테이블 준비
df_mapping = load_mapping_table()
    
# 2. DB 연결
conn = dbconnect.MydbConnect('seoul_urban_lab')
    
total_processed = 0
    
# 3. 파일별 순차 처리 (2022 -> 2023 -> 2024)
for file_path in TARGET_FILES:
            
    # 변환
    df_final = process_file(file_path, df_mapping)
    print(f"변환 완료: {len(df_final)}행")
        
    # 적재 (10000개씩 끊어서 넣기 - 메모리 보호)
    CHUNK_SIZE = 10000
    file_inserted_count = 0
        
    print("DB 적재 시작")
    for i in range(0, len(df_final), CHUNK_SIZE):
        chunk = df_final.iloc[i:i+CHUNK_SIZE]
        cnt = insert_to_db(conn, chunk)
        file_inserted_count += cnt
        print(f"      - {i + cnt} / {len(df_final)} 완료", end="\r")
            
    print(f"\n    {os.path.basename(file_path)} 적재 완료! (+{file_inserted_count}건)")
    total_processed += file_inserted_count
        
conn.close()
print("\n" + "="*50)
print(f"모든 작업 완료! 총 {total_processed}건의 데이터가 DB에 저장되었습니다.")
print("="*50)
